<a href="https://colab.research.google.com/github/nkinsman16/HW-Week-3-BasicCleaning_R/blob/main/BasicCleaning_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning: recurring coding strategies

Imagine we have this data:

In [1]:
IRdisplay::display_html('<iframe width="700" height="300" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pubhtml?gid=0&single=true"></iframe>')

And you need to create a cleaner version:

In [2]:
IRdisplay::display_html('<iframe width="700" height="300" src="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pubhtml?gid=2024244899&single=true"></iframe>')

The actual cleaning plan starts after exploration, and the  strategies to recover the true value of the cell can be one or several of these:

* **keep** the columns or rows needed
* **replace** the wrong value for good value.
* **delete** the wrong value.
* **extract** the good value.
* **split** the cell contents


**It is also recommended to clean the column names before the cell contents.**

Notice the **column names** have lightblue background,  and the **contents** have a grey background. I have also colored in red the column names that may serve as key columns, the unique identifiers.


## 1. Some REGEX

REGEX is **complex**, but let's share some _patterns_ that we will often use.

* This [a-zA-Z] represents any character of the alphabet (based in latin alphabet).
* This [^a-zA-Z] represents any characters *outside* the alphabet. The [^] means "the opposite" here. You can use for other purposes (see below).
* This \w is not the same as [a-zA-Z], \w can be equal to [0-9a-zA-Z\s_], that is, it includes numbers, spaces (\s) and underscore (_).
* Then, \W is the opposite of \w
* Similarly, \d is equal to [0-9], and \D is the opposite.
* Some other relevant symbols are:
  - The dot (**.**), representing any character.
  - The plus (**+**), here \d+ represents one or more digits.
  - The asterisk (__*__) here [a-z]* represents zero or more lowercase letters.
  - The symbols **^** and **\$** are also very relevant. Together the represent a whole string, that is,  ^\d[a-z]$ means the string starts with a digit and ends with a lowercase letter.

  You will need these when exploring and implementing the cleaning.

## 2. The Data Types

Imagine you CAN NOT see all the data. In that case we can use some code.

In [3]:
# the link as CSV
linkToData="https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ubcCBaveg-58jcVmbErpO5kZswjFyHN5YlB8tB1a8B4fzU4sqZ08jkOKx4kBz1qtDNkJJWH8vBYF/pub?gid=0&single=true&output=csv"

Read the data:

In [4]:
dirty=read.csv(linkToData)

I recommend you do this first:

In [5]:
str(dirty)

'data.frame':	9 obs. of  7 variables:
 $ X  : chr  "identification" "Perú" "USA" "Canada" ...
 $ X.1: chr  "identification2" "Peru, South America" "USA, North America" "Canada, North America" ...
 $ X.2: logi  NA NA NA NA NA NA ...
 $ X.3: chr  "var1" "1,500.00" "2,500.00" "3,500.00" ...
 $ X.4: chr  "var 2" "1'200" "1'300" "--" ...
 $ X.5: chr  "var@3" "500" "$1 500" "1.5" ...
 $ X.6: chr  "category" "a" "A" "Ba" ...


You should see the column names, but you do not ...because the column names are in the wrong place:

In [6]:
#using 'head'
head(dirty)

,X,X.1,X.2,X.3,X.4,X.5,X.6
,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
1,identification,identification2,NA,var1,var 2,var@3,category
2,Perú,"Peru, South America",NA,"1,500.00",1'200,500,a
3,USA,"USA, North America",NA,"2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America",NA,"3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa",NA,"2,500.00",,_,Ba
6,Israel [note],"Israel [note], Asia",NA,Dk,250k,-,?


Using _head_ and _tail_ is important:

In [7]:
tail(dirty)

,X,X.1,X.2,X.3,X.4,X.5,X.6
,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
4,Canada,"Canada, North America",NA,"3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa",NA,"2,500.00",,_,Ba
6,Israel [note],"Israel [note], Asia",NA,Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe",NA,"2,550.00",310 000,330,Ba
8,,,NA,,,,
9,source: IDK,,NA,,,,


Notice empty cells have not been considered "NA". We could write instead:

In [8]:
dirty=read.csv(linkToData,na.strings = c(""))
dirty

X,X.1,X.2,X.3,X.4,X.5,X.6
<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
identification,identification2,NA,var1,var 2,var@3,category
Perú,"Peru, South America",NA,"1,500.00",1'200,500,a
USA,"USA, North America",NA,"2,500.00",1'300,$1 500,A
Canada,"Canada, North America",NA,"3,500.00",--,1.5,Ba
Côte D'Ivoire,"Côte D'Ivoire, Africa",NA,"2,500.00",NA,_,Ba
Israel [note],"Israel [note], Asia",NA,Dk,250k,-,?
United Kingdom,"United Kingdom, Europe",NA,"2,550.00",310 000,330,Ba
NA,NA,NA,NA,NA,NA,NA
source: IDK,NA,NA,NA,NA,NA,NA


We are now in a situation when we beliveve we have a better idea of what rows/columns are needed.

# Cleaning Steps

## A. Keeping columns needed

Pay attention to column names:
- They should be on the top.
- They should have no spaces anywhere
- They should have no special characters
- They should have no start with numeric values
- They should be readable (short, self-explanatory)
- When needed, use comments to explain their meaning.

Currently the column names are somewhere they should not:

---



In [9]:
#current names:
names(dirty)

[1] "X"   "X.1" "X.2" "X.3" "X.4" "X.5" "X.6"

In [10]:
# They are in the contents
as.list(dirty[1,]) # always use list

$X
[1] "identification"

$X.1
[1] "identification2"

$X.2
[1] NA

$X.3
[1] "var1"

$X.4
[1] "var 2"

$X.5
[1] "var@3"

$X.6
[1] "category"

In [11]:
# renaming
colnames(dirty) <- as.list(dirty[1, ])

# remove first row (now it's the header)
dirty <- dirty[-1, ]

## see new column names
dirty

,identification,identification2,NA,var1,var 2,var@3,category
,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America",NA,"1,500.00",1'200,500,a
3,USA,"USA, North America",NA,"2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America",NA,"3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa",NA,"2,500.00",NA,_,Ba
6,Israel [note],"Israel [note], Asia",NA,Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe",NA,"2,550.00",310 000,330,Ba
8,NA,NA,NA,NA,NA,NA,NA
9,source: IDK,NA,NA,NA,NA,NA,NA


This data has columns with no values at all, even the column name is missing, we can use that to our advantage:

In [12]:
names(dirty)[names(dirty)!='NA']

[1] "identification"  "identification2" "var1"            "var 2"          
[5] "var@3"           "category"

In [13]:
# then
# using index labels
dirty <-   dirty[, names(dirty)[names(dirty)!='NA']]
# so
dirty

,identification,identification2,var1,var 2,var@3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America","1,500.00",1'200,500,a
3,USA,"USA, North America","2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America","3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa","2,500.00",NA,_,Ba
6,Israel [note],"Israel [note], Asia",Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe","2,550.00",310 000,330,Ba
8,NA,NA,NA,NA,NA,NA
9,source: IDK,NA,NA,NA,NA,NA


## B. Keeping rows needed on the contents

### B.1 Filtering using index positions

In [14]:
dirty[1:6,] # this may seem confusing

,identification,identification2,var1,var 2,var@3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America","1,500.00",1'200,500,a
3,USA,"USA, North America","2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America","3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa","2,500.00",NA,_,Ba
6,Israel [note],"Israel [note], Asia",Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe","2,550.00",310 000,330,Ba


In [15]:
head(dirty,-2) # this works.. easy to understand?

,identification,identification2,var1,var 2,var@3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America","1,500.00",1'200,500,a
3,USA,"USA, North America","2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America","3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa","2,500.00",NA,_,Ba
6,Israel [note],"Israel [note], Asia",Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe","2,550.00",310 000,330,Ba


### B.2 Filtering using a column with missing values:

In [16]:
dirty[!is.na(dirty$identification2),]

,identification,identification2,var1,var 2,var@3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America","1,500.00",1'200,500,a
3,USA,"USA, North America","2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America","3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa","2,500.00",NA,_,Ba
6,Israel [note],"Israel [note], Asia",Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe","2,550.00",310 000,330,Ba


Let's keep the first option:

In [17]:
dirty= dirty[1:6,]

## C. Exploring cell contents

Identify which are textual, numerical, or categorical.

In [18]:
str(dirty)

'data.frame':	6 obs. of  6 variables:
 $ identification : chr  "Perú" "USA" "Canada" "Côte D'Ivoire" ...
 $ identification2: chr  "Peru, South America" "USA, North America" "Canada, North America" "Côte D'Ivoire, Africa" ...
 $ var1           : chr  "1,500.00" "2,500.00" "3,500.00" "2,500.00" ...
 $ var 2          : chr  "1'200" "1'300" "--" NA ...
 $ var@3          : chr  "500" "$1 500" "1.5" "_" ...
 $ category       : chr  "a" "A" "Ba" "Ba" ...




* Columns **identification1** and **identification2** are *textual*.
* The columns from **var1** to **var@3** are all *numerical*. But if the type is _chr_ the column has been read as strings.
* Column **category** is *categorical*. Keep in mind that categorical types will NEVER be recognised as such by default when read from a CSV. They will always be understood as text (_chr_).

The **column names** are always *strings*.


### C.2.1. **Exploring TEXT columns**

When data is textual, you need to explore the cells to verify all the characters are part of the **alphabet**.


Let's see how to use R:

In [19]:
# show me the cells that have a character outside the alphabet
dirty$identification[grepl("[^a-zA-Z]", dirty$identification)]


[1] "Perú"           "Côte D'Ivoire"  "Israel [note]"  "United Kingdom"

United Kingdom is not dirty. But the space is outside the alphabet. What about:

In [20]:
dirty[grepl("\\W", dirty$identification), 'identification']

[1] "Côte D'Ivoire"  "Israel [note]"  "United Kingdom"

or...

In [21]:
# this looks ok
dirty$identification[grep("[^a-zA-Z\\s]",dirty$identification,perl = T)]

[1] "Perú"          "Côte D'Ivoire" "Israel [note]"

### C.2.2. **Exploring NUMBERS**

If numbers are recognised as so, there is no cleaning needed. But if not, it means it has been recognised as text, then we use the regex **\d** (and its variations):

In [22]:
dirty$var1[grep("\\D",dirty$var1)]

[1] "1,500.00" "2,500.00" "3,500.00" "2,500.00" "Dk"       "2,550.00"

In [23]:
dirty[,'var 2'][grep("\\D",dirty$'var 2')]

[1] "1'200"   "1'300"   "--"      "250k"    "310 000"

In [25]:
### remember you do not want this:
# dirty[,'var 2'][grep("\\D",dirty$'var 2')] |>as.numeric()

Notice I need to use **[  ]** to access the variables with dirty names (space between words, and the **@** special character). That is why you clean the column names first:

In [26]:
dirty[,'var@3'][grep("\\D",dirty$'var@3',perl=T)]

[1] "$1 500" "1.5"    "_"      "-"

There are cells with good values, but other values can not be kept. Use **\D** with care, numbers are complex. So I prefer something like this:

In [27]:
dirty[,'var@3'][grep("[^\\d+\\.*\\d*]", dirty$'var@3', perl=T,invert = F)]

[1] "$1 500" "_"      "-"

### C.2.3. **Exploring CATEGORICAL columns**

Just prepare a fequency table to see if categories are well defined.

In [28]:
table(dirty$category)


 ?  a  A Ba 
 1  1  1  3 

### C.2.4. Exploring Column names

Here, we want to see what is not right.

In [29]:
# names that have characters different than numbers or alphabet
colnames(dirty)[grepl("[^0-9a-zA-Z]", colnames(dirty))]

[1] "var 2" "var@3"

## C. Cleaning

As mentioned, cleaning may mean:

a. Making bad characters disappear.

b. Keeping good characters.


Let's start with the _column names_:

### C.1 Cleaning column names

How can you say: if "a space" or a "weird character", disappear? (that is, *replace* by *nothing*)


In [30]:
# option 1
gsub("\\W",'',names(dirty) , perl=T )

[1] "identification"  "identification2" "var1"            "var2"           
[5] "var3"            "category"

In [31]:
# option 2
gsub("[^\\w]",'',names(dirty), perl=T )

[1] "identification"  "identification2" "var1"            "var2"           
[5] "var3"            "category"

In [32]:
# # option 3
gsub("[^0-9a-zA-Z]",'',names(dirty), perl=T )

[1] "identification"  "identification2" "var1"            "var2"           
[5] "var3"            "category"

Choose any and make the change:

In [33]:
names(dirty)=gsub("[^0-9a-zA-Z]",'',names(dirty))
dirty

,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America","1,500.00",1'200,500,a
3,USA,"USA, North America","2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America","3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa","2,500.00",NA,_,Ba
6,Israel [note],"Israel [note], Asia",Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe","2,550.00",310 000,330,Ba


Be preventive about leading and trailing spaces:

In [34]:
colnames(dirty) == trimws(colnames(dirty),whitespace = "[\\h\\v]")

[1] TRUE TRUE TRUE TRUE TRUE TRUE

The column names were cleaned by **Making bad characters disappear** 🙂

### C.2. Cleaning TEXT columns

Let's check the **identification** column:

In [35]:
dirty$identification[grep("[^a-zA-Z\\s]",dirty$identification,perl = T)]

[1] "Perú"          "Côte D'Ivoire" "Israel [note]"

Not all cells have characters that are not in the alphabet;BUT THIS TIME, The **only** problem here is the brackets.

Then:

* Option 1: Whatever inside brackets (including the brackets) has to go!

In [36]:
gsub("\\[.*\\]",'',dirty$identification,perl = T)

[1] "Perú"           "USA"            "Canada"         "Côte D'Ivoire" 
[5] "Israel "        "United Kingdom"

* Option 2: Splitting

In [37]:
sapply(strsplit(dirty$identification, split = '[', fixed = TRUE), `[`, 1)

[1] "Perú"           "USA"            "Canada"         "Côte D'Ivoire" 
[5] "Israel "        "United Kingdom"

When you are satisfied, make the change:

In [38]:
dirty$identification <- gsub("\\[.*\\]", "", dirty$identification)
dirty

,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,"Peru, South America","1,500.00",1'200,500,a
3,USA,"USA, North America","2,500.00",1'300,$1 500,A
4,Canada,"Canada, North America","3,500.00",--,1.5,Ba
5,Côte D'Ivoire,"Côte D'Ivoire, Africa","2,500.00",NA,_,Ba
6,Israel,"Israel [note], Asia",Dk,250k,-,?
7,United Kingdom,"United Kingdom, Europe","2,550.00",310 000,330,Ba


The **splitting** option seems very convenient for **identification2**:

In [39]:
sapply(strsplit(dirty$identification2, split = ',', fixed = TRUE), `[`, 2)

[1] " South America" " North America" " North America" " Africa"       
[5] " Asia"          " Europe"

If this is OK, then:

In [40]:
dirty$identification2 <- sapply(strsplit(dirty$identification2, split = ',', fixed = TRUE), `[`, 2)
dirty

,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Perú,South America,"1,500.00",1'200,500,a
3,USA,North America,"2,500.00",1'300,$1 500,A
4,Canada,North America,"3,500.00",--,1.5,Ba
5,Côte D'Ivoire,Africa,"2,500.00",NA,_,Ba
6,Israel,Asia,Dk,250k,-,?
7,United Kingdom,Europe,"2,550.00",310 000,330,Ba


Be preventive about leading and trailing spaces:

In [41]:
dirty$identification == trimws(dirty$identification)

[1]  TRUE  TRUE  TRUE  TRUE FALSE  TRUE

Check!

In [42]:
dirty$identification

[1] "Perú"           "USA"            "Canada"         "Côte D'Ivoire" 
[5] "Israel "        "United Kingdom"

Then:

In [43]:
dirty$identification <- trimws(dirty$identification)

In [44]:
#verifying:
dirty$identification == trimws(dirty$identification)

[1] TRUE TRUE TRUE TRUE TRUE TRUE

The presence of ortographic symbols might complicate things. What about?

In [47]:
## you may need to install this:
#install.packages(string)

In [45]:
# load the stringi package
library(stringi)

# remove accents and special orthographic characters from 'identification'
stri_trans_general(dirty$identification, "Latin-ASCII")

[1] "Peru"           "USA"            "Canada"         "Cote D'Ivoire" 
[5] "Israel"         "United Kingdom"

In [48]:
# then

dirty$identification <- stringi::stri_trans_general(dirty$identification, "Latin-ASCII")

### C.3. Cleaning the CAT column

We had this:

In [49]:
table(dirty$category)


 ?  a  A Ba 
 1  1  1  3 

You can conclude that the **a** is wrong, it should be **A**.

In [50]:
#what about:
gsub('a','A', dirty$category,fixed=T)

[1] "A"  "A"  "BA" "BA" "?"  "BA"

That changed **Ba** to **BA**!

In [51]:
## maybe
## ^: start of string
## $: end  of string
gsub('^a$','A', dirty$category)

[1] "A"  "A"  "Ba" "Ba" "?"  "Ba"

In [52]:
#then
dirty$category <- gsub("^a$", "A", dirty$category)
dirty

,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Peru,South America,"1,500.00",1'200,500,A
3,USA,North America,"2,500.00",1'300,$1 500,A
4,Canada,North America,"3,500.00",--,1.5,Ba
5,Cote D'Ivoire,Africa,"2,500.00",NA,_,Ba
6,Israel,Asia,Dk,250k,-,?
7,United Kingdom,Europe,"2,550.00",310 000,330,Ba



### C.4. Cleaning NUM columns

From the previous exploration, we know some issues in these columns. Let's go step by step:

In [53]:
gsub(',','',dirty$var1)


[1] "1500.00" "2500.00" "3500.00" "2500.00" "Dk"      "2550.00"

Then,

In [54]:
dirty$var1=gsub(',','',dirty$var1)
dirty


,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Peru,South America,1500.00,1'200,500,A
3,USA,North America,2500.00,1'300,$1 500,A
4,Canada,North America,3500.00,--,1.5,Ba
5,Cote D'Ivoire,Africa,2500.00,NA,_,Ba
6,Israel,Asia,Dk,250k,-,?
7,United Kingdom,Europe,2550.00,310 000,330,Ba


The **var2** is more complicated.

In [55]:
dirty$var2

[1] "1'200"   "1'300"   "--"      NA        "250k"    "310 000"

Let me create a NEW variable as a flag:

In [56]:
# save where you have the issue
dirty$var2_temp=grepl("\\'|k",dirty$var2,fixed=F)
dirty

,identification,identification2,var1,var2,var3,category,var2_temp
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
2,Peru,South America,1500.00,1'200,500,A,TRUE
3,USA,North America,2500.00,1'300,$1 500,A,TRUE
4,Canada,North America,3500.00,--,1.5,Ba,FALSE
5,Cote D'Ivoire,Africa,2500.00,NA,_,Ba,FALSE
6,Israel,Asia,Dk,250k,-,?,TRUE
7,United Kingdom,Europe,2550.00,310 000,330,Ba,FALSE


In [57]:
## now replace
dirty$var2 <- gsub("'|k|\\s", "", dirty$var2)
dirty

,identification,identification2,var1,var2,var3,category,var2_temp
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
2,Peru,South America,1500.00,1200,500,A,TRUE
3,USA,North America,2500.00,1300,$1 500,A,TRUE
4,Canada,North America,3500.00,--,1.5,Ba,FALSE
5,Cote D'Ivoire,Africa,2500.00,NA,_,Ba,FALSE
6,Israel,Asia,Dk,250,-,?,TRUE
7,United Kingdom,Europe,2550.00,310000,330,Ba,FALSE


The last value was right. The other ones were simplified (lack '000'). Let's put the '000' back!

In [58]:
# just adding 000 where needed
ifelse(dirty$var2_temp,paste0(dirty$var2,'000'),dirty$var2)


[1] "1200000" "1300000" "--"      NA        "250000"  "310000"

In [59]:
# then
dirty$var2=ifelse(dirty$var2_temp,paste0(dirty$var2,'000'),dirty$var2)
# and delete this
dirty$var2_temp=NULL
# result so far
dirty


,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Peru,South America,1500.00,1200000,500,A
3,USA,North America,2500.00,1300000,$1 500,A
4,Canada,North America,3500.00,--,1.5,Ba
5,Cote D'Ivoire,Africa,2500.00,NA,_,Ba
6,Israel,Asia,Dk,250000,-,?
7,United Kingdom,Europe,2550.00,310000,330,Ba


The **var3** can be solved like this:

In [60]:
dirty['var3']=gsub("\\$|\\s",'',dirty$var3)
dirty

,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Peru,South America,1500.00,1200000,500,A
3,USA,North America,2500.00,1300000,1500,A
4,Canada,North America,3500.00,--,1.5,Ba
5,Cote D'Ivoire,Africa,2500.00,NA,_,Ba
6,Israel,Asia,Dk,250000,-,?
7,United Kingdom,Europe,2550.00,310000,330,Ba


## D. Coding missing values:


Wrong missing values representation should be replaced with care. Do it according to the data type.

Then, let's start with the **categorical** column:

In [61]:
badCats=grep('\\W+',dirty$category,value = T)
badCats

[1] "?"

Let's go for the **numerical** cases:

In [62]:
badNums1 <- dirty$var1[!grepl("\\d+.*\\d*", dirty$var1)]
badNums1

[1] "Dk"

In [63]:
badNums2 <- dirty$var2[!grepl("\\d+.*\\d*", dirty$var2)]
badNums2

[1] "--" NA

In [64]:
badNums3 <- dirty$var3[!grepl("\\d+.*\\d*", dirty$var3)]
badNums3

[1] "_" "-"

In [65]:
unique(c(badCats, badNums1, badNums2, badNums3))

[1] "?"  "Dk" "--" NA   "_"  "-"

Or we can create a function:

In [66]:
 detect_WrongNumber <- function(x) {
  x[!grepl("\\d+.*\\d*", x)]
  }


bads <- lapply(dirty[, c("var1", "var2", "var3")], detect_WrongNumber)
all_bads <- unique(unlist(bads))
all_bads <- union(all_bads, badCats)
all_bads



[1] "Dk" "--" NA   "_"  "-"  "?"

Let's recode all the **all_bads** into **NA**:

In [67]:
# Let's recode all the **all_bads** into **NA**:
dirty[] <- data.frame(lapply(dirty, function(x) replace(x, x %in% all_bads, NA)))

Are we missing something?

## E. The Resetting of indexes

After many changes, you have this:

In [68]:
dirty

,identification,identification2,var1,var2,var3,category
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2,Peru,South America,1500.00,1200000,500,A
3,USA,North America,2500.00,1300000,1500,A
4,Canada,North America,3500.00,NA,1.5,Ba
5,Cote D'Ivoire,Africa,2500.00,NA,NA,Ba
6,Israel,Asia,NA,250000,NA,NA
7,United Kingdom,Europe,2550.00,310000,330,Ba


In general, you want to be sure the row index starts in 'one' and continues with consecutive values. Just do this:

In [69]:
rownames(dirty)=NULL
print(dirty)

  identification identification2    var1    var2 var3 category
1           Peru   South America 1500.00 1200000  500        A
2            USA   North America 2500.00 1300000 1500        A
3         Canada   North America 3500.00    <NA>  1.5       Ba
4  Cote D'Ivoire          Africa 2500.00    <NA> <NA>       Ba
5         Israel            Asia    <NA>  250000 <NA>     <NA>
6 United Kingdom          Europe 2550.00  310000  330       Ba


In [70]:
dirty

identification,identification2,var1,var2,var3,category
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Peru,South America,1500.00,1200000,500,A
USA,North America,2500.00,1300000,1500,A
Canada,North America,3500.00,NA,1.5,Ba
Cote D'Ivoire,Africa,2500.00,NA,NA,Ba
Israel,Asia,NA,250000,NA,NA
United Kingdom,Europe,2550.00,310000,330,Ba


# SAVING the CLEAN data

In [71]:
write.csv(dirty,"nowClean.csv")


In [72]:
nowClean=dirty[,]


folder <- "dataCleaned"

# Check if the folder exists
if (!dir.exists(folder)) {
  # Create the folder
  dir.create(folder)
  write.csv(nowClean,file.path(folder,"nowClean.csv"))

} else {
  write.csv(nowClean,file.path(folder,"nowClean.csv"))}
